# Naive Bayes Classifier 

## 1) Traning set 만들기

In [9]:
# training set, test set 만들기
import pandas as pd

data_set = pd.read_csv('./data/model/training_set.csv')
training_set = data_set[:int(len(data_set)*0.9)]
test_set = data_set[int(len(data_set)*0.9):]

## 2) Traning

In [12]:
# traning : traning set 으로 토큰별 빈도수 계산
from collections import defaultdict

# 토큰별로 문서내 빈도수 카운팅
wordfreq = defaultdict(lambda : [0, 0])
for i, row in training_set.iterrows():
    words = row['ngrams in doc'].split(',')
    for word in words :
        if row['lable'] == 1 :
            wordfreq[word][1] += 1
        elif row['lable'] == -1 :
            wordfreq[word][0] += 1
    
    if i % 10000 == 0 :
        print("{} / {}".format(i, training_set.shape[0]))

0 / 229561
10000 / 229561
20000 / 229561
30000 / 229561
40000 / 229561
50000 / 229561
60000 / 229561
70000 / 229561
80000 / 229561
90000 / 229561
100000 / 229561
110000 / 229561
120000 / 229561
130000 / 229561
140000 / 229561
150000 / 229561
160000 / 229561
170000 / 229561
180000 / 229561
190000 / 229561
200000 / 229561
210000 / 229561
220000 / 229561


In [13]:
#ngram list 파일 저장
import pandas as pd

pd.DataFrame.from_dict(wordfreq, orient='index').to_csv('./data/model/ngram_list.csv')

In [24]:
#dictionary 파일 저장
import pandas as pd
import numpy as np

k = 0.5

df = pd.read_csv('./data/model/ngram_list.csv', index_col='Unnamed: 0')
df["freq"] = df['0']+df['1'] # 전체 토큰별 빈도수 계산
df = df.loc[df.freq > 15] # 빈도가 15 초과인것만 필터링
df["w|pos"] = (df['1']+k)/(df['1'].sum()+k*2)
df["w|neg"] = (df['0']+k)/(df['0'].sum()+k*2)
df["log(w|pos)"] = np.log(df["w|pos"])
df["log(w|neg)"] = np.log(df["w|neg"])
df["polarity score"] = df["w|pos"]/df["w|neg"]
df["intensity"] = [x if x > 1 else 1/x for x in df['polarity score']]
df["lable"] = [1 if x > 1 else -1 for x in df['polarity score']]
df.loc[df.intensity>1.3].to_csv('./data/model/bok_dictionary.csv')

In [33]:
import pandas as pd

df = pd.read_csv('./data/model/bok_dictionary.csv', index_col='Unnamed: 0')
dic = df.to_dict('index')

## 3) prediction & 검증

In [92]:
import math

prob_pos_tot = math.log(df['1'].sum()/(df['1'].sum()+df['0'].sum()))
prob_neg_tot = math.log(df['0'].sum()/(df['1'].sum()+df['0'].sum()))

TP = 0
FN = 0
FP = 0
TN = 0

cnt = 0

predict=[]

for i, row in test_set[(test_set.lable==1.0) | (test_set.lable==-1.0)].iterrows():
    ngrams = row['ngrams in doc'].split(',')
    lable = row['lable']
    prob_predict = 0
    prob_pos = 0
    prob_neg = 0
    
    for token in ngrams :
        if token in dic.keys() :
            prob_pos += dic[token]['log(w|pos)']
            prob_neg += dic[token]['log(w|neg)']
    prob_pos = math.exp(prob_pos + prob_pos_tot)
    prob_neg = math.exp(prob_neg + prob_neg_tot)
    
    if prob_pos+prob_neg == 0 :
        prob_predict = 0
    else :
        prob_predict = prob_pos/(prob_pos+prob_neg)
        
    if prob_predict > 0.5 and lable == 1.0 :
        predict.append(1.0)
        TP += 1
    elif prob_predict > 0.5 and lable == -1.0 :
        predict.append(1.0)
        FP += 1
    elif prob_predict < 0.5 and lable == 1.0 :
        predict.append(-1.0)
        FN += 1
    elif prob_predict < 0.5 and lable == -1.0 :
        predict.append(-1.0)
        TN += 1
    


In [83]:
def r_squared(y_true, y_hat):
    ssr = 0
    sst = 0
    e = np.subtract(y_true, y_hat)
    y_mean = np.mean(y_true)
    for item in e:
        ssr += item**2
    for item in y_true:
        sst += (item - y_mean)**2
    r2 = 1 - ssr / sst
    return r2

In [91]:
r_squared(lable_ls, predict)

-0.8554504743825104

In [96]:
#ACC = (TP + TN)  / (전체 데이타 수 = P + N)
ACC = (TP + TN) / (TP + TN + FP + FN)
print("ACC = {}".format(ACC))
# ERR = (FN+FP) / (전체 데이타수 = P+N)
ERR = (FN+FP) / (TP + TN + FP + FN)
print("ERR = {}".format(ERR))
# SN = (TP) / P
SN = (TP) / (TP + FN)
print("SN = {}".format(SN))
# PREC = TP / (TP+FP)
PREC = TP / (TP+FP)
print("PREC = {}".format(PREC))

ACC = 0.5518053375196232
ERR = 0.44819466248037676
SN = 0.7241640617599697
PREC = 0.6006914433880726
